In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

# -----------------------------
# 1. Load Gold price data
# -----------------------------
gold = yf.download("GC=F", start="2015-01-01")

# IMPORTANT: Flatten multi-level columns (Colab / yfinance fix)
gold.columns = gold.columns.get_level_values(0)

gold = gold[['High', 'Low', 'Close']]

# -----------------------------
# 2. Calculate True Range (TR)
# -----------------------------
gold['Prev_Close'] = gold['Close'].shift(1)

gold['TR1'] = gold['High'] - gold['Low']
gold['TR2'] = (gold['High'] - gold['Prev_Close']).abs()
gold['TR3'] = (gold['Low'] - gold['Prev_Close']).abs()

gold['TR'] = gold[['TR1', 'TR2', 'TR3']].max(axis=1)

# -----------------------------
# 3. Calculate ATR (14-day)
# -----------------------------
ATR_PERIOD = 14
gold['ATR'] = gold['TR'].rolling(ATR_PERIOD).mean()

# -----------------------------
# 4. ATR percentile (volatility normalization)
# -----------------------------
WINDOW = 252  # ~1 trading year

gold['ATR_percentile'] = (
    gold['ATR']
    .rolling(WINDOW)
    .apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
)

# -----------------------------
# 5. Volatility regime classification
# -----------------------------
def atr_regime(p):
    if p < 0.20:
        return 'LOW'
    elif p < 0.70:
        return 'NORMAL'
    elif p < 0.90:
        return 'HIGH'
    else:
        return 'EXTREME'

gold['Vol_Regime'] = gold['ATR_percentile'].apply(atr_regime)

# -----------------------------
# 6. System state (ACTIVE / INACTIVE)
# -----------------------------
def system_state(vol_regime):
    if vol_regime in ['NORMAL', 'HIGH']:
        return 'ACTIVE'
    else:
        return 'INACTIVE'

gold['System_State'] = gold['Vol_Regime'].apply(system_state)

# -----------------------------
# 7. Final check (for verification)
# -----------------------------
gold[['Close', 'ATR', 'ATR_percentile', 'Vol_Regime', 'System_State']].tail(10)


/tmp/ipython-input-745925218.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  gold = yf.download("GC=F", start="2015-01-01")
[*********************100%***********************]  1 of 1 completed


Price,Close,ATR,ATR_percentile,Vol_Regime,System_State
Date,,,,,
2026-01-23,4976.200195,72.792864,0.825397,HIGH,ACTIVE
2026-01-26,5079.700195,72.099993,0.809524,HIGH,ACTIVE
2026-01-27,5079.899902,68.878523,0.750000,HIGH,ACTIVE
2026-01-28,5301.600098,82.364223,0.900794,EXTREME,INACTIVE
2026-01-29,5318.399902,114.178537,0.984127,EXTREME,INACTIVE
2026-01-30,4713.899902,164.171422,1.000000,EXTREME,INACTIVE
2026-02-02,4622.500000,187.464251,1.000000,EXTREME,INACTIVE
2026-02-03,4903.700195,210.578544,1.000000,EXTREME,INACTIVE
2026-02-04,4920.399902,220.471436,1.000000,EXTREME,INACTIVE
